In [1]:
import numpy as np
from deepsudoku import REPO_PATH
import os
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import tqdm

os.chdir(REPO_PATH)

In [2]:
wanted = ['eval/avg_episodic_length',
  'eval/avg_episodic_return',
  'eval/avg_winrate',
  'charts/avg_episodic_return',
  'charts/avg_episodic_length',]


def get_df(path):
  event_acc = EventAccumulator(path)
  event_acc.Reload()
  event_acc.Tags()

  df = pd.DataFrame([], columns = ["time_stamp"]+wanted)


  for scalar in wanted:

    for val in event_acc.Scalars(scalar):
      
      
      df.loc[val.step,["time_stamp",scalar]] = [val.wall_time,val.value]
  
  path = Path(path).name
  env = path[:path.find("_")]
  df["env"] = env
  number = path.find("_") + 5
  name = path[path.find("_")+2:path.find("__",number)]
  df["run"] = name
  return df    

dfs = []

for path in tqdm.tqdm(glob.glob("train_best_model/runs/*")):

  dfs.append(get_df(path))

  0%|          | 0/4 [00:00<?, ?it/s]

2023-09-07 12:07:58.177446: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 12:07:58.740247: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
100%|██████████| 4/4 [01:18<00:00, 19.71s/it]


In [3]:
data = pd.concat(dfs)
data = data.reset_index(names ="step")
data.to_csv("stats/rl_best_model_training.csv")

In [4]:
data

,step,time_stamp,eval/avg_episodic_length,eval/avg_episodic_return,eval/avg_winrate,charts/avg_episodic_return,charts/avg_episodic_length,env,run
0,58578944,1693558521.854788,17.139999,16.844999,0.41,NaN,NaN,Sudoku-nostop0,AnealLRMuchLonger
1,58630144,1693558540.308062,17.51,17.26,0.5,78.277779,29.722221,Sudoku-nostop0,AnealLRMuchLonger
2,58681344,1693558561.554642,15.53,15.22,0.38,NaN,NaN,Sudoku-nostop0,AnealLRMuchLonger
3,58732544,1693558581.823086,17.940001,17.68,0.48,NaN,NaN,Sudoku-nostop0,AnealLRMuchLonger
4,58783744,1693558600.218876,15.45,15.16,0.42,NaN,NaN,Sudoku-nostop0,AnealLRMuchLonger
...,...,...,...,...,...,...,...,...,...
41019,28878848,1693513675.559044,NaN,NaN,NaN,76.545456,31.454546,Sudoku-nostop0,AnealLR
41020,28879872,1693513675.891061,NaN,NaN,NaN,76.633331,31.366667,Sudoku-nostop0,AnealLR
41021,28880896,1693513676.223087,NaN,NaN,NaN,74.967743,33.032257,Sudoku-nostop0,AnealLR
41022,28881920,1693513676.555005,NaN,NaN,NaN,77.333336,30.666666,Sudoku-nostop0,AnealLR
